In [2]:
import numpy as np
import pandas as pd
import json
from datetime import datetime

## 게임 결과 데이터 프레임 만들기


입력해야 할 것 : API에서 추출한 게임 결과 데이터(df), 현재 게임 버젼(version), 라이엇에서 제공하는 data_dragon


### object 설명

1. df : 데이터프레임

2. version : 게임 버전

3. data_dragon : data dragon

4. item_data : data dragon에서 추출한 아이템, 증강체 정보

5. champ : data dragon에서 추출한 유닛 정보

6. game_id : 해당 게임 아이디

7. game_result : 해당 게임 결과

8. trait_language : 특성 한/영 변환 정보 - 해당 정보는 data dragon에 존재해서, 추후 수정 예정

9. champ_language : 유닛 한/영 변환 정보

19. date : 해당 게임 일자



### 함수 설명

* change_augment_name : 증강체 이름을 수정해서, game_result dataframe에 augment1, augment2, augment3 column으로 생성해주는 함수

* make_trait_dataframe(traits) : game_result의 traits column에 속한 element를 입력, 해당 유저의 경기 결과 중 특성 정보 제공해주는 함수

* change_language_trait(traits) : make_trait_dataframe로 만든 데이터프레임을 입력, 특성 정보를 한/영 변환해주는 함수


* make_unit_dataframe(unit) : game_result의 units columns에 속한 element를 입력, 해당 유저의 경기 결과 중 유닛 정보 제공해주는 함수 (유닛 id, 유닛 이름, 유닛 코스트, 유닛 성수, 착용아이템1, 착용아이템2, 착용아이템3), 한영 변환까지 진행한 결과를 제공해줍니다.

* make_trait_result(number) : number에 해당하는 플레이어의 특성 종합 정보를 제공해주는 함수

* make_unit_result(number) : number에 해당하는 플레이어의 유닛 종합 정보를 제공해주는 함수


* make_new_game_result : 해당 경기 데이터를 데이터프레임으로 정리해주는 함수. 결과 데이터프레임은 다음과 같습니다.

    1. Datetime : 해당 게임 플레이 일자.
    2. Game_id : 해당 게임 id
    3. Player_id : player 정보를 puuid로 제공
    4. Placement : 등수,
    5. Time : 해당 게임 플레이 시간.
    6. Last_round : 게임이 끝났을 때의 마지막 라운드.
    7. Damage_player : 해당 유저가 다른 유저에게 가한 데미지. 
    8. Gold_left : 게임이 끝났을 때 남은 골드
    9. augment1, augment2, augment3 : 증강체. 각각 2-1, 3-2, 4-2에 선택한 증강체
    10. traits : 최종 조합 특성 정보
        10-1. num_units : 해당 특성을 가진 유닛 수. list로 존재하며, traits에 속하는 list 길이와 같습니다.
        10-2. tier_current : 현재 티어 레벨. list로 존재하며, traits에 속하는 list 길이와 같습니다.
    11. units : 최종 조합 유닛 정보
        11-1. unit_tier : 해당 유닛의 성(star)수. list로 존재하며, units에 속하는 list 길이와 같습니다.
        11-2. item_name : 해당 유닛이 착용한 아이템. list로 존재하며, units에 속한 list 길이와 같습니다.


    traits에 관련된 column에 대한 해석은 다음과 같습니다.

        * traits column은 해당 유저가 사용한 특성 이름이 차례대로 나열된 list가 삽입된다.
        * traits column에 속한 특성 list의 i번째 특성에 해당하는 유닛 수는 num_units에 속한 list의 i번 째 element가 된다.
        * traits column 에 속한 특성 list의 i번째 특성에 해당하는 특성 티어는 tier_current에 속한 list의 i번 째 element가 된다.


    units에 관련된 column에 대한 해석은 다음과 같습니다.

        * units column은 해당 유저가 사용한 유닛의 이름이 차례대로 나열된 list가 삽입된다.
        * units column에 속한 유닛 list의 i번 째 유닛의 성(star)수는 unit_tier에 속한 list의 i번 째 element가 된다.
        * units column에 속한 유닛 list의 i번 째 유닛이 착용한 아이템은 item_name에 속한 list의 i번 째 element가 된다.(이 때, i번 째 element는 길이가 3인 리스트이다.)

In [11]:
file_path='./item data.json'
with open(file_path, "r") as json_file:
    data_dragon = json.load(json_file)
    


In [1]:
class make_df:
    
    def __init__(self, df, version, data_dragon):
        self.df = df
        self.version = version
        self.data_dragon = data_dragon
        self.item_data = pd.DataFrame(self.data_dragon['items'])
        self.champ = pd.DataFrame(pd.DataFrame(self.data_dragon['setData']).iloc[2, 0])
        self.game_id = self.df['metadata']['match_id']
        self.game_result = pd.DataFrame(self.df['info']['participants']).drop(['companion'], axis=1)
        self.game_result['game_id'] = self.game_id
        self.game_result = self.game_result[['game_id', 'puuid', 'placement', 'level', 'gold_left', 'last_round', 'time_eliminated',
                                             'total_damage_to_players', 'augments', 'traits', 'units']]
        self.trait_language = pd.DataFrame({'Korean' : ['민간인', '자동방어체계', '동물특공대', '레이저단', '싸움꾼', '메카:프라임', '위협',
                                                '마스코트', '기상캐스터', '에이스', '결투가', '황소부대', '병기고', '방패대',
                                                '지하세계', '별 수호자', '익살꾼', '선의', '타락', '우세', '무법자', '주문투척자',
                                                '정찰단', '특등사수', '엄호대', '기계유망주', '해커'],
                                   "English" : ['Set8_Civilian', 'Set8_Admin', 'Set8_AnimaSquad', 'Set8_SpaceCorps', 'Set8_Brawler',
                                                 'Set8_ExoPrime', 'Set8_Threat', 'Set8_Mascot', 'Set8_Forecaster', 'Set8_Ace',
                                                 'Set8_Duelist', 'Set8_OxForce', 'Set8_Arsenal', 'Set8_Aegis', 'Set8_UndergroundThe',
                                                 'Set8_StarGuardian', 'Set8_Prankster', 'Set8_Heart', 'Set8_Corrupted', 'Set8_Supers',
                                                 'Set8_Renegade', 'Set8_Channeler', 'Set8_Recon', 'Set8_Deadeye', 'Set8_Defender',
                                                'Set8_GenAE', 'Set8_Hacker']})
        self.champ_language = {self.champ.apiName[i] : self.champ.name[i] for i in range(self.champ.shape[0])}
        self.date= str(datetime.fromtimestamp(int(self.df['info']['game_datetime'])/1000)).split(' ')[0]
        
    # 증강체 한글 변환
    
    def change_augment_name(self):
    
    
        change_results =[]

        for augments in self.game_result['augments']:

            change_result = []

            for augment in augments:

                change_augment = self.item_data.loc[self.item_data.apiName == augment, 'name'].values[0]
                change_result.append(change_augment)
            
            length_change_result = len(change_result)
            
            if length_change_result<3:
                
                change_result = change_result + ['None']*(3-length_change_result)

            change_results.append(change_result)
            
            
        change_results = pd.DataFrame(change_results, columns=['augment1', 'augment2', 'augment3'])
        
        

        self.game_result[['augment1', 'augment2', 'augment3']] = change_results
        self.game_result = self.game_result[['game_id', 'puuid', 'placement', 'level', 'gold_left',
                                            'last_round', 'time_eliminated', 'total_damage_to_players',
                                             'augment1', 'augment2', 'augment3', 'traits', 'units']]

        return self.game_result
    
    
    # trait dataframe 만들기
    
    
    def make_trait_dataframe(self,trait):
    
        trait = pd.DataFrame(trait)
        
        if trait.shape[0]==0:
            return pd.DataFrame(columns = ['name', 'num_units', 'tier_current', 'tier_total'])

        trait = trait[trait.tier_current!=0].drop(['style'], axis=1).reset_index(drop=True)

        return trait
    
    

    def change_language_trait(self,trait):

        name = []

        for i in trait['name']:
            change_name = self.trait_language.loc[self.trait_language['English']==i, 'Korean'].values
            
            if len(change_name)==0:
                name.append('None')
            else:
                
                name.append(change_name[0])

        trait['name_kor'] = name
    
        return trait

    
    # Unit dataframe 만들기
    
    def make_unit_dataframe(self,unit):
    
        units = pd.DataFrame(unit)
        
        if units.shape[0]==0:
            return pd.DataFrame(columns = ['character_id', 'champ_name', 'rarity', 'tier', 'items', 'item1', 'item2', 'item3'])
        

        item_list = []
        champ_name = []
        for i in range(len(units['character_id'])):
            
            champ_name.append(self.champ_language[units['character_id'][i]])
            check = units['itemNames'][i]

            check += [0]*(3-len(check))

            item_list.append(check)

        units['items'] = item_list

        item1 = []
        item2 = []
        item3 = []

        for item in item_list:

            check_item = []

            for i in item:
                if i==0:
                    check_item.append('None')
                else:
                    item_kor = self.item_data.loc[self.item_data.apiName==i, 'name'].values[0]
                    check_item.append(item_kor)
            item1.append(check_item[0])
            item2.append(check_item[1])
            item3.append(check_item[2])


        units = pd.DataFrame(unit).drop(['itemNames', 'name'], axis=1)

        units['item1'] = item1
        units['item2'] = item2
        units['item3'] = item3
        units.character_id = list(map(lambda x: x.replace(self.version, ''), units.character_id))
        units['champ_name'] = champ_name

        return units
    
    
    # 수정된 데이터프레임 만들기
    
    
    def make_new_game_result(self):
    

        New_result = {'Datetime' : self.date,
            'Game_id' : self.df['metadata']['match_id'],
                      'Player_id' : [],
                 'Placement' : [], 
                 'Time' : [],
                 'Last_round' : [],
                 "Damage_player" : [],
                 "Gold_left" : [],
                 "augment1" : [],
                  'augment2' : [],
                  'augment3' : [],
                 "traits" : [],
                 "num_units" : [],
                 "tier_current" : [],
                 "units" : [],
                 "unit_tier" : [],
                 "item_name" : []}

        New_result['Player_id'] += (list(self.game_result.puuid))
        
        New_result['Placement'] +=(list(self.game_result.placement))

        New_result['Time']+=(list(self.game_result.time_eliminated))

        New_result['Last_round']+=(list(self.game_result.last_round))

        New_result['Damage_player']+=(list(self.game_result.total_damage_to_players))

        New_result['Gold_left']+=(list(self.game_result.gold_left))

        New_result['augment1'] += list(self.game_result.augment1)
        
        New_result['augment2'] += list(self.game_result.augment2)

        New_result['augment3'] += list(self.game_result.augment3)
        
        

        for player in range(self.game_result.shape[0]):
            


            trait_dataframe = pd.DataFrame(self.game_result.loc[player, 'traits'])
            trait_dataframe = self.make_trait_dataframe(trait_dataframe)
            trait_dataframe = self.change_language_trait(trait_dataframe)

            New_result['traits'].append(list(trait_dataframe.name_kor))
            New_result['num_units'].append(list(trait_dataframe.num_units))
            New_result['tier_current'].append(list(trait_dataframe.tier_current))


                
            unit_dataframe =  self.make_unit_dataframe(self.game_result.loc[player, 'units'])
            

            New_result['units'].append(list(unit_dataframe.champ_name))
            New_result['unit_tier'].append(list(unit_dataframe.tier))
            New_result['item_name'].append(np.array(unit_dataframe.loc[:, ['item1', 'item2', 'item3']]).tolist())


        New_result = pd.DataFrame(New_result)

        return pd.DataFrame(New_result)
    
    
    # 실제 method 입력시 trait dataframe을 제공해주는 함수
    
    def make_trait_result(self, number):
        
        trait1 = self.make_trait_dataframe(self.game_result.loc[number, 'traits'])
        trait1 = self.change_language_trait(trait1)
        trait1['Datetime'] = self.date
        trait1['Game_id'] = self.game_id
        trait1['Player_id'] = self.game_result.loc[number, 'puuid']
        trait1 = trait1[['Datetime', 'Game_id', 'Player_id', 'name', 'num_units', 'tier_current', 'tier_total', 'name_kor']]
        
        return trait1
    
    # 실제 method 입력시 unit dataframe을 제공해주는 함수
    
    def make_unit_result(self, number):
        
        unit1 = self.make_unit_dataframe(self.game_result.loc[number, 'units'])
        unit1['Datetime'] = self.date
        unit1['Game_id'] = self.game_id
        unit1['Player_id'] = self.game_result.loc[number, 'puuid']
        unit1 = unit1[['Datetime', 'Game_id', 'Player_id', 'character_id', 'champ_name', 'rarity', 'tier', 'item1', 'item2', 'item3']]
        return unit1

